In [ ]:
import pandas as pd

path = r"C:\Users\ElifYilmaz\OneDrive - Enpal B.V\Desktop\New folder\Project\reroute, more samples\merged_df_bigger_sample.csv"
df = pd.read_csv(path)
df.columns = df.columns.str.lower()


# optionally turn it into net sc only 
df = df [ df ['net_sc'] != 0].copy()

#separating into old positives, new positives and all negatives nb: would be better to mark also the older/ newer negatives maybe later
new_positives = df[ (df['old'] == 0) & (df['grosscontractsigned']== 1)].reset_index(drop=True)
old_positives =  df[df['old']==1].reset_index(drop=True)
rest_negatives = df[ df[ 'grosscontractsigned'] == 0].reset_index(drop=True)

negative_train = rest_negatives.sample(frac=0.7, random_state=42)
negative_test = rest_negatives.drop(negative_train.index)

# these is 4184. to keep the original ratio i have to add 210 new positives
test_new_positives = new_positives.sample(n=244, random_state = 42)
train_new_positives = new_positives.drop(test_new_positives.index)

# create final train and test
train = pd.concat( [negative_train, train_new_positives, old_positives])
test = pd.concat ( [test_new_positives, negative_test])

# boo dont forget to drop old mark
train = train.drop('old', axis=1)
test = test.drop('old', axis=1)

# for some reason selfipadone is missing on a few columns, later look but for now fill with 0 
train['selfipa_done'] = train['selfipa_done'].fillna(0)
test['selfipa_done'] = test['selfipa_done'].fillna(0)

#split 
X_train = train.drop('grosscontractsigned', axis=1)
y_train = train['grosscontractsigned']
X_test = test.drop('grosscontractsigned', axis=1)
y_test = test['grosscontractsigned']
